In [1]:

import cv2 as cv
import math
import numpy as np
from matplotlib import pyplot as plt
import random as rng
import glob
from IPython.display import Image, display

sift = cv.SIFT_create()
bf1 = cv.BFMatcher()
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
orb = cv.ORB_create()



flann = cv.FlannBasedMatcher()

In [2]:
imgData = {
  "kp": [],
  "d": [],
  "image": []
}

matchData = {
  "matches": [],
    'matrix':[],
  "score": [],
}


pairData = {
  "idx": 0,
  "score": 1000,
  "matchdata":matchData,
    "matrix1":[],
    "matrix2":[]
}

info = {
  "matches":[],
    "score":[],
    "matrix":[],
      "imgdata": [],
  "pairs": [],
    'total':0
}

In [3]:
def getKeyp(folder):
    d=[]
    kp=[]
    img=[]
    xa=[]
    a=0
    print('finding data...')
    il=[]
    for imageName in glob.glob(folder+'/*.jpg'):
        a=a+1
        i1 = cv.imread(imageName)
        kp1, d1 = orb.detectAndCompute(i1,None)#key points and detectors
        imgData['kp']=kp1
        imgData['d']=d1
        imgData['image']=cv.imread(imageName)
        img.append(imgData.copy())
    for i in range(a):
        
        xa.append([])
        info['pairs'].append([])
    for i in range(a):
        info['matrix'].append(xa.copy())
        info['score'].append(xa.copy())
        info['matches'].append(xa.copy())
    info['total']=a
    print('data found')
#     print(imgData)
#     print(info)
    info['imgdata']=img
    return info.copy()

In [4]:
def getData(d1,d2,k1,k2,i,j):
    s=0
    qu=[]
    tr=[]
    kp1=[]
    kp2=[]

    good_points =[]
    matches=[]
    matches = bf.match(d1,d2)
#     for m,n in allMatch:
#         if(m.distance < 0.75*n.distance):
#             matches.append(m)
#     if len(matches)==0:
#         print("oof")
#         if(m.distance < 0.85*n.distance):
#             matches.append(m)
    orderedmatches = sorted(matches, key = lambda x:x.distance)
    numTosum=int(len(matches)/10)
    if numTosum<1:
        numTosum=1
    if numTosum%2:
        numTosum=numTosum-1
    for m in orderedmatches[:numTosum]:
        s=s+m.distance
    
    for m in range(len(matches)):
        thresh=50
        if 2*orderedmatches[0].distance>thresh:
            thresh=2*orderedmatches[0].distance
        if matches[m].distance <= thresh and len(good_points)<30:
#             print(matches[m])
            good_points.append(matches[m])
#             print(good_points)
#     if(len(good_points)<4):
#         kp1=[]
#         kp2=[]
#         good_points =[]
#         for m in range(len(matches)):
#             if m.distance <= thresh*2 and len(good_points)<60/4:
#                 good_points.append(matches[m])
#     print(good_points.copy())
#     print( info['matches'][i][j])
    info["matches"][i][j]=good_points.copy()
    info["matches"][j][i]=good_points.copy()
    good_points=[]
    info["score"][i][j]=(s/numTosum)
    info["score"][j][i]=(s/numTosum)
#     print( info["score"][i])
#     print(" ")
#     print(" ")
#     print(info['matches'][i][j])
#     print(info)
#     print(s/numTosum)
    return s/numTosum

In [5]:
def getPairs(folder):
    info=getKeyp(folder)
    print(info)
    data=[]
    pairs=[]
    pair=[]
    pair1=[]
    matchData=[]
    thresh=20
    for i in range(0,info['total']-1):
        for j in range(i+1,info['total']):
            score= getData(info['imgdata'][i]["d"],info['imgdata'][j]["d"],info['imgdata'][i]["kp"],info['imgdata'][j]["kp"],i,j)
#             print(info["score"])
            info["score"][i][j]=score
            info["score"][j][i]=score
#             print(info["score"])
            #             print(matchData["score"])
#             office:20
#             james:25    
            if folder == 'office':
                thresh=20
            else:
                thresh=30
#             print(info["score"][i][j])
            if(score<thresh):
#                 pair.append(j)
#                 pair1.append(i)
                info['pairs'][i].append(j)
                info['pairs'][j].append(i)
        pair=[]
        pair1=[]
    return info.copy()

In [6]:
def  getMat():
    print("start mat finding...")
    mat = []
    tempmat = []
    M=[]
    for p in range(info["total"]):
        if len(info["pairs"][p]) ==0:
            print("***ERROR***")
        for i in range(len(info["pairs"][p])):
            xq=info["pairs"][p][i]
#             print(p)
#             print(xq)
#             print(info['matrix'][p][xq])

#             if len(info['matrix'][p][xq])>0:
#                 continue
#             print(info['matches'][p][info['pairs'][p][i]])
            p1 = np. float32([info['imgdata'][p]['kp'][m.queryIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)
            p2 = np. float32([info['imgdata'][info["pairs"][p][i]]['kp'][m.trainIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)
            M, mask = cv.findHomography(p2, p1, cv.RANSAC,5.0)        
            info['matrix'][p][info["pairs"][p][i]] = M.copy()
            M=[]
    return

In [7]:
def stitch(i,j,done,carry):
    j=info['pairs'][i][j]
    done.append(i)
    result=[]
    skip = 0
#     if i>=len(info["pairs"]) or info["pairs"][i][j]>=len(info["pairs"]) or len(info["pairs"][i])==0:
#         done.append(info["pairs"][i][j])
#         print("*")
#         return [],0
    queryImg=info["imgdata"][i]["image"].copy()
    trainImg=info["imgdata"][j]["image"].copy()
    print(j)
    
#     print(len(info["pairs"][i]))
#     for ix in done:
#         if info["pairs"][i][j] == ix:
#             skip=1
#     if skip == 0:        
#         carry=stitch(info,info["pairs"][i][j],0,done,[])[0]
#     if len(carry)>0:
#         trainImg=carry
    width = trainImg.shape[1] + queryImg.shape[1]
    height = trainImg.shape[0] + queryImg.shape[0]
#     width = 10000
#     height=10000
    print(info["matrix"][i][j])
    print(info["matrix"][j][i])
    result = cv.warpPerspective(trainImg,info["matrix"][i][j] , (width, height))
    result[0:queryImg.shape[0], 0:queryImg.shape[1]] = queryImg

    plt.figure(figsize=(20,10))
    plt.imshow(result)

# plt.axis('off')
    plt.show()
    return result,info["pairs"][i][j]

In [ ]:

info = getPairs("james")

finding data...


In [ ]:
print("start matrix")
getMat()
print("done matrix")

In [ ]:
a=0
x=stitch(a,0,[],[])

In [ ]:
x=stitch(a,0,[],[])

In [ ]:
i=0
img1=info['imgdata'][i]['image']
kp1=info['imgdata'][i]['kp']
pair=info['pairs'][i][0]
img2=info['imgdata'][pair]['image']
kp2=info['imgdata'][pair]['kp']
print(pair)
matches=info['matches'][i][pair]
print(len(matches))
img3 = cv.drawMatches(img1,kp1,img2,kp2,matches,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(15,7))
plt.imshow(img3),plt.show()